In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_quote(soup):
    # Make the soup
    # soup = BeautifulSoup(text)
    
    # Locate target elements from the soup
    quote_text = soup.find('span')
    author = soup.find('small')
    author_url = soup.find('a')
    tags = soup.find_all('a', {'class': 'tag'})

    # Extract information from the retrieved elements
    quote_text = quote_text.text
    author = author.text
    author_url = author_url.get('href')
    tags_list = []
    for tag in tags:
        tags_list.append(tag.text)
    
    # Tidy up the retrieved outputs
    author_url = 'http://quotes.toscrape.com' + author_url
    tags = ';'.join(tags_list)
    
    # Store the results
    results_dictionary = {
      'quote_text': quote_text,
      'author': author,
      'author_url': author_url,
      'tags': tags
    }
    
    return results_dictionary

In [3]:
def get_quotes(page_number):
    # define the URL
    base_url = 'http://quotes.toscrape.com/page/'
    page_number = str(page_number)
    url = base_url + page_number
    
    # make a request to retrieve HTML codes and make the soup
    r = requests.get(url)
    c = r.content
    soup = BeautifulSoup(c, 'lxml')
    
    # locate all the quotes
    quotes = soup.find_all('div', {'class': 'quote'})
    
    # parse each quote using for loops
    outputs = []
    for quote in quotes:
        quote = get_quote(quote)
        outputs.append(quote)
        
    # return the outputs
    return outputs

In [4]:
def scrape_quotes(start, end):
    outputs = []
    for i in list(range(start, end + 1)):
        outputs += get_quotes(i)
    outputs = pd.DataFrame(outputs)
    return outputs

In [5]:
outputs = scrape_quotes(1, 10)

In [6]:
outputs.head()

,author,author_url,quote_text,tags
0,Albert Einstein,http://quotes.toscrape.com/author/Albert-Einstein,“The world as we have created it is a process ...,change;deep-thoughts;thinking;world
1,J.K. Rowling,http://quotes.toscrape.com/author/J-K-Rowling,"“It is our choices, Harry, that show what we t...",abilities;choices
2,Albert Einstein,http://quotes.toscrape.com/author/Albert-Einstein,“There are only two ways to live your life. On...,inspirational;life;live;miracle;miracles
3,Jane Austen,http://quotes.toscrape.com/author/Jane-Austen,"“The person, be it gentleman or lady, who has ...",aliteracy;books;classic;humor
4,Marilyn Monroe,http://quotes.toscrape.com/author/Marilyn-Monroe,"“Imperfection is beauty, madness is genius and...",be-yourself;inspirational


In [7]:
outputs.to_csv('quotes_extracted.csv')